## Trajnet++ challenge
Before running:  
Delete the test folder, duplicate test_original folder and rename it to test

In [10]:
import generative_model
import numpy as np
import math

### Generative model

In [2]:
params = {
    "NOISE": 0.1,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.8,
    
    "STOP_PROB": 0.2,
    
    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,
    
    "VELOCITY_CHANGE_PROB": 0.2,
    "VELOCITY_CHANGE_NOISE": 0.1,
    
    "ANGLE_CHANGE_PROB": 0.05,
    "ANGLE_CHANGE_NOISE": 1,
    
    "CLUSTER_PERCENTAGES": [0.2, 0.68, 0.95, 1.0]
}

In [3]:
import json
import pandas as pd

In [39]:
import trajnetplusplustools
from trajnetplusplustools.data import TrackRow
test_path = 'trajnet_data/test_original/'
pred_path = 'trajnet_data/test/'
file_paths = [
    'real_data/biwi_eth',
    'real_data/crowds_uni_examples',
    'real_data/crowds_zara02',
    'synth_data/collision_test',
    'synth_data/orca_synth'
]
sample = 1.0

In [41]:
for file_path in file_paths:

    reader = trajnetplusplustools.Reader(test_path + file_path + '.ndjson', scene_type='paths')
    scenes = [(s_id, s) for s_id, s in reader.scenes(sample=sample)]

    for scene_data in scenes:
        scene_id = scene_data[0]
        pedestrian_tracks = scene_data[1]

        primary_pedestrian_track = pedestrian_tracks[0]
        pedestrian_id = primary_pedestrian_track[0].pedestrian
        last_frame_no = primary_pedestrian_track[-1].frame

        xs = []
        ys = []
        for frame in primary_pedestrian_track:
            frame_id = frame.frame
            x = frame.x
            y = frame.y

            xs.append(x)
            ys.append(y)

        all_pred_x, all_pred_y, weights = generative_model.predict(xs, ys, params, trajectory_length=12)
        sorting_order = np.array(weights).argsort()[::-1]
        all_pred_x = np.array(all_pred_x)[sorting_order]
        all_pred_y = np.array(all_pred_y)[sorting_order]

        for prediction_no in range(len(all_pred_x)):
            pred_x = all_pred_x[prediction_no]
            pred_y = all_pred_y[prediction_no]

            for i in range(len(pred_x)):
                pred_row = TrackRow(
                    frame=last_frame_no + i + 1,
                    pedestrian=pedestrian_id,
                    x=pred_x[i],
                    y=pred_y[i],
                    prediction_number=prediction_no,
                    scene_id=scene_id
                )

                pred_row = trajnetplusplustools.writers.trajnet_tracks(pred_row)
                with open(pred_path + file_path + '.ndjson', "a") as pred_file:
                    pred_file.write(pred_row + '\n')
                                 

### Avg vel + rot with scenarios

In [3]:
import trajnetplusplustools
from trajnetplusplustools.data import TrackRow
test_path = 'trajnet_data/test_original/'
pred_path = 'trajnet_data/test/'
file_paths = [
    'real_data/biwi_eth',
    'real_data/crowds_uni_examples',
    'real_data/crowds_zara02',
    'synth_data/collision_test',
    'synth_data/orca_synth'
]
sample = 1.0

In [6]:
def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def avg_vel_and_rot_const(sample_x, sample_y, noise_var=0.1):
    sample_vel_x = [(sample_x[i] - sample_x[i-1]) + np.random.normal(0, noise_var) for i in range(1, len(sample_x))]
    sample_vel_y = [(sample_y[i] - sample_y[i-1]) + np.random.normal(0, noise_var) for i in range(1, len(sample_y))]
    
    all_angles = []
    for i in range(1, len(sample_vel_x)):
        prev_vel = [sample_vel_x[i-1], sample_vel_y[i-1]]
        curr_vel = [sample_vel_x[i], sample_vel_y[i]]
        one_angle = np.math.atan2(np.linalg.det([prev_vel, curr_vel]),np.dot(prev_vel, curr_vel))   
        all_angles.append(one_angle)
    #print(all_angles)
    
    avg_angle = np.mean(all_angles)
    
    avg_vel_x = np.mean(sample_vel_x)
    avg_vel_y = np.mean(sample_vel_y)
    
    pred_x = [sample_x[-1] + i*avg_vel_x for i in range(1, 6)]
    pred_y = [sample_y[-1] + i*avg_vel_y for i in range(1, 6)]

    rot_pred_x = [pred_x[0]]
    rot_pred_y = [pred_y[0]]
    # the following could be replaced with the rotate() method
    rot_pred_x[0] = sample_x[-1] + math.cos(avg_angle) * (rot_pred_x[0] - sample_x[-1]) - math.sin(avg_angle) * (rot_pred_y[0] - sample_y[-1])
    rot_pred_y[0] = sample_y[-1] + math.sin(avg_angle) * (rot_pred_x[0] - sample_x[-1]) + math.cos(avg_angle) * (rot_pred_y[0] - sample_y[-1])
    # redefine the average velocity as it now has a new heading
    avg_vel_x = rot_pred_x[0] - sample_x[-1]
    avg_vel_y = rot_pred_y[0] - sample_y[-1]
    for i in range(1, 12):

        rot_pred_x.append(rot_pred_x[-1] + avg_vel_x)
        rot_pred_y.append(rot_pred_y[-1] + avg_vel_y)
        
        prev_x = rot_pred_x[i-1]
        cur_x = rot_pred_x[i]
        prev_y = rot_pred_y[i-1]
        cur_y = rot_pred_y[i]
        
        rot_x, rot_y = rotate((prev_x, prev_y), (cur_x, cur_y), avg_angle)
        
        rot_pred_x[i] = rot_x
        rot_pred_y[i] = rot_y
        
        # redefine the average velocity as it now has a new heading
        avg_vel_x = rot_x - prev_x
        avg_vel_y = rot_y - prev_y
    
    return rot_pred_x, rot_pred_y

In [19]:
for file_path in file_paths:

    reader = trajnetplusplustools.Reader(test_path + file_path + '.ndjson', scene_type='paths')
    scenes = [(s_id, s) for s_id, s in reader.scenes(sample=sample)]

    for scene_data in scenes:
        scene_id = scene_data[0]
        pedestrian_tracks = scene_data[1]

        primary_pedestrian_track = pedestrian_tracks[0]
        pedestrian_id = primary_pedestrian_track[0].pedestrian
        last_frame_no = primary_pedestrian_track[-1].frame

        xs = []
        ys = []
        for frame in primary_pedestrian_track:
            frame_id = frame.frame
            x = frame.x
            y = frame.y

            xs.append(x)
            ys.append(y)

        all_pred_x, all_pred_y = [], []
        for i in range(3):
            pred_x, pred_y = avg_vel_and_rot_const(xs, ys)
            all_pred_x.append(pred_x)
            all_pred_y.append(pred_y)

        for prediction_no in range(len(all_pred_x)):
            pred_x = all_pred_x[prediction_no]
            pred_y = all_pred_y[prediction_no]

            for i in range(len(pred_x)):
                pred_row = TrackRow(
                    frame=last_frame_no + i + 1,
                    pedestrian=pedestrian_id,
                    x=pred_x[i],
                    y=pred_y[i],
                    prediction_number=prediction_no,
                    scene_id=scene_id
                )

                pred_row = trajnetplusplustools.writers.trajnet_tracks(pred_row)
                with open(pred_path + file_path + '.ndjson', "a") as pred_file:
                    pred_file.write(pred_row + '\n')
                                 